In [24]:
# Lab 8_1 code:

In [26]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, TimestampType, IntegerType, FloatType
from time import sleep

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("SparkStreaming")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

dataSchema = StructType(
        [StructField("ts", TimestampType(), True),
         StructField("generation_solar", IntegerType(), True),
         StructField("generation_wind_offshore", IntegerType(), True),
         StructField("generation_wind_onshore", IntegerType(), True),
         StructField("forecast_solar_day_ahead", IntegerType(), True),
         StructField("forecast_wind_onshore_day_ahead", IntegerType(), True),
         StructField("price_day_ahead", FloatType(), True),
         StructField("price_actual", FloatType(), True)
         ])

# Read from a source 
sdf = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1) \
        .csv("/home/jovyan/data/activity-data")
print(sdf)
# Do a calculation
activityCounts = sdf.groupBy("price_day_ahead").count()

# Write to a sink - here, the output is memory (only for testing). The query name (i.e., activity_counts) will be the Spark SQL table name.
activityQuery = activityCounts.writeStream.queryName("price_day_ahead_counts") \
                    .format("memory").outputMode("complete") \
                    .start()
# Testing 
for x in range(10):
    spark.sql("SELECT * FROM price_day_ahead_counts").show()
    sleep(5)

DataFrame[ts: timestamp, generation_solar: int, generation_wind_offshore: int, generation_wind_onshore: int, forecast_solar_day_ahead: int, forecast_wind_onshore_day_ahead: int, price_day_ahead: float, price_actual: float]
+---------------+-----+
|price_day_ahead|count|
+---------------+-----+
+---------------+-----+

+---------------+-----+
|price_day_ahead|count|
+---------------+-----+
+---------------+-----+

+---------------+-----+
|price_day_ahead|count|
+---------------+-----+
+---------------+-----+

+---------------+-----+
|price_day_ahead|count|
+---------------+-----+
+---------------+-----+

+---------------+-----+
|price_day_ahead|count|
+---------------+-----+
+---------------+-----+

+---------------+-----+
|price_day_ahead|count|
+---------------+-----+
+---------------+-----+

+---------------+-----+
|price_day_ahead|count|
+---------------+-----+
|         4271.0|    1|
|          714.0|    1|
|         5138.0|    1|
|         7408.0|    3|
|         1033.0|    1|
|  

In [27]:
# Stop the spark context
spark.stop()

In [25]:
# Lab 8_2 code

In [9]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, TimestampType, IntegerType, FloatType
from time import sleep

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("SparkStreaming")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

dataSchema = StructType(
        [StructField("ts", TimestampType(), True),
         StructField("generation_solar", IntegerType(), True),
         StructField("generation_wind_offshore", IntegerType(), True),
         StructField("generation_wind_onshore", IntegerType(), True),
         StructField("forecast_solar_day_ahead", IntegerType(), True),
         StructField("forecast_wind_onshore_day_ahead", IntegerType(), True),
         StructField("price_day_ahead", FloatType(), True),
         StructField("price_actual", FloatType(), True)
         ])

# Read from a source 
sdf = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1) \
        .csv("/home/jovyan/data/activity-data")

print(sdf)
# Do a calculation
activityCounts = sdf.groupBy("price_day_ahead").count()

# Write to a sink - here, the output is memory (only for testing). The query name (i.e., activity_counts) will be the Spark SQL table name.
activityQuery = activityCounts.writeStream.queryName("price_day_ahead_counts") \
                    .format("console").outputMode("complete") \
                    .start()
try:
    activityQuery.awaitTermination()
except KeyboardInterrupt:
    activityQuery.stop()
    # Stop the spark context
    spark.stop()
    print("Stopped the streaming query and the spark context")

DataFrame[ts: timestamp, generation_solar: int, generation_wind_offshore: int, generation_wind_onshore: int, forecast_solar_day_ahead: int, forecast_wind_onshore_day_ahead: int, price_day_ahead: float, price_actual: float]


Py4JJavaError: An error occurred while calling o455.start.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.C

In [42]:
# Stop the spark context
spark.stop()

In [43]:
# Lab 8_3 batches to big query

# Laat deze een tijdje runnen en daarna stoppen, dan is het te zien in Google Cloud bij Big Query

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, TimestampType, IntegerType, FloatType
from time import sleep


sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("SparkStreaming")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# We need to set the following configuration whenever we need to use GCS.
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "de_jads_temp_snellejassie"
spark.conf.set('temporaryGcsBucket', bucket)

dataSchema = StructType(
        [StructField("ts", TimestampType(), True),
         StructField("generation_solar", IntegerType(), True),
         StructField("generation_wind_offshore", IntegerType(), True),
         StructField("generation_wind_onshore", IntegerType(), True),
         StructField("forecast_solar_day_ahead", IntegerType(), True),
         StructField("forecast_wind_onshore_day_ahead", IntegerType(), True),
         StructField("price_day_ahead", FloatType(), True),
         StructField("price_actual", FloatType(), True)
         ])

# Read from a source 
sdf = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1) \
        .csv("/home/jovyan/data/activity-data/activity-data-p1")
print(sdf)
# .option("maxBytesPerTrigger", "65536b")
# Do a calculation
activityCounts = sdf.groupBy("price_actual", "generation_solar", "generation_wind_offshore", "generation_wind_onshore"
                                           , "forecast_solar_day_ahead", "forecast_wind_onshore_day_ahead", "price_day_ahead").count()

def my_foreach_batch_function(df, batch_id):
   # Saving the data to BigQuery as batch processing sink -see, use write(), save(), etc.
    df.write.format('bigquery') \
      .option('table', 'de2022-assignment2-fresh.energyprediction.energytable12') \
      .mode("overwrite") \
      .save()

# Write to a sink - here, the output is written to a Big Query Table
# Use your gcp bucket name. 
# ProcessingTime trigger with two-seconds micro-batch interval
activityQuery = activityCounts.writeStream.outputMode("complete") \
                    .trigger(processingTime = '2 seconds').foreachBatch(my_foreach_batch_function).start()
try:
    activityQuery.awaitTermination()
except KeyboardInterrupt:
    activityQuery.stop()
    # Stop the spark context
    spark.stop()
    print("Stopped the streaming query and the spark context")

DataFrame[ts: timestamp, generation_solar: int, generation_wind_offshore: int, generation_wind_onshore: int, forecast_solar_day_ahead: int, forecast_wind_onshore_day_ahead: int, price_day_ahead: float, price_actual: float]


In [3]:
# Stop the spark context
spark.stop()
